In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [3]:
import pandas as pd
from src.config import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / "tabular_data.parquet")

In [4]:
from datetime import datetime

from src.data_utils import split_time_series_data

X_train, y_train, X_test, y_test = split_time_series_data(
    df,
    cutoff_date=datetime(2023, 9, 1, 0, 0, 0),
    target_column="target"
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(55900, 674)
(55900,)
(31720, 674)
(31720,)


In [5]:
past_ride_columns = [c for c in X_train.columns if c.startswith("rides_")]
X_train_only_numeric = X_train[past_ride_columns]
X_test_only_numeric = X_test[past_ride_columns]

In [6]:
pip install lightgbm


Note: you may need to restart the kernel to use updated packages.


In [7]:
import lightgbm as lgb
model = lgb.LGBMRegressor()
model.fit(X_train_only_numeric, y_train)

C:\Users\peddi\anaconda3\envs\sp25_taxi\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\peddi\anaconda3\envs\sp25_taxi\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\peddi\anaconda3\envs\sp25_taxi\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\peddi\anaconda3\envs\sp25_taxi\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\peddi\anaconda3\envs\sp25_taxi\Lib\subprocess

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.104503 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 159651
[LightGBM] [Info] Number of data points in the train set: 55900, number of used features: 672
[LightGBM] [Info] Start training from score 11.110286


LGBMRegressor()

In [8]:
from sklearn.metrics import mean_absolute_error
predictions = model.predict(X_test_only_numeric)

In [9]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")

3.3054


In [10]:
import mlflow
import dagshub

In [11]:
#import mlflow
#import dagshub

# Initialize DagsHub MLflow tracking
dagshub.init(
    repo_owner="Shreyas559955",  # Your DagsHub username
    repo_name="spr25_taxi",  # Your repository name
    mlflow=True  # Enable MLflow tracking
)

# Set MLflow tracking URI to your DagsHub repository
mlflow.set_tracking_uri("https://dagshub.com/Shreyas559955/spr25_taxi.mlflow")

# Verify connection
print("MLflow Tracking URI:", mlflow.get_tracking_uri())


Accessing as Shreyas9265

Initialized MLflow to track repo "Shreyas559955/spr25_taxi"

Repository Shreyas559955/spr25_taxi initialized!

MLflow Tracking URI: https://dagshub.com/Shreyas559955/spr25_taxi.mlflow


In [12]:
#import mlflow

#def get_mlflow_client():
   # try:
   #     mlflow.set_tracking_uri('https://dagshub.com/Shreyas559955/sp25_taxi.mlflow')
        # Test a simple request to check if the server is reachable
    #    mlflow.get_experiment_by_name('sp25_taxi')
    #    return mlflow
    #except Exception as e:
     #   print(f"Failed to connect to remote MLflow, error: {e}")
     #   mlflow.set_tracking_uri('https://dagshub.com/Shreyas559955/sp25_taxi.mlflow')  # Fallback to local
     #   return mlflow

#mlflow = get_mlflow_client()


In [13]:
from src.experiment_utils import set_mlflow_tracking, log_model_to_mlflow
from dotenv import load_dotenv
import os
load_dotenv() 

mlflow = set_mlflow_tracking()
log_model_to_mlflow(model, X_test_only_numeric, "LGBMRegressor", "mean_absolute_error", score=test_mae)

INFO:src.experiment_utils:MLflow tracking URI and credentials set.
ERROR:src.experiment_utils:An error occurred while logging to MLflow: API request to https://dagshub.com/Shreyas559955/spr25_taxi.mlflow/api/2.0/mlflow/experiments/get failed with exception HTTPSConnectionPool(host='dagshub.com', port=443): Max retries exceeded with url: /Shreyas559955/spr25_taxi.mlflow/api/2.0/mlflow/experiments/get?experiment_id=None (Caused by ResponseError('too many 500 error responses'))


MlflowException: API request to https://dagshub.com/Shreyas559955/spr25_taxi.mlflow/api/2.0/mlflow/experiments/get failed with exception HTTPSConnectionPool(host='dagshub.com', port=443): Max retries exceeded with url: /Shreyas559955/spr25_taxi.mlflow/api/2.0/mlflow/experiments/get?experiment_id=None (Caused by ResponseError('too many 500 error responses'))

Tuning Learning Rate

In [14]:
import lightgbm as lgb
import mlflow
import mlflow.sklearn
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV

# Initialize MLflow experiment
mlflow.set_experiment("LightGBM_Tuning")

# Define learning rate values to test
learning_rates = [0.01, 0.05, 0.1, 0.2, 0.5]
best_lr = None
best_mae = float("inf")

print("Step 1: Tuning Learning Rate...")
for lr in learning_rates:
    with mlflow.start_run():
        model = lgb.LGBMRegressor(learning_rate=lr)
        model.fit(X_train_only_numeric, y_train)
        y_pred = model.predict(X_test_only_numeric)
        mae = mean_absolute_error(y_test, y_pred)

        # Log results to MLflow
        mlflow.log_param("learning_rate", lr)
        mlflow.log_metric("mae", mae)

        print(f"Learning Rate: {lr}, MAE: {mae}")

        # Store the best learning rate
        if mae < best_mae:
            best_mae = mae
            best_lr = lr

print(f"Best Learning Rate: {best_lr} with MAE: {best_mae}")


MlflowException: API request to https://dagshub.com/Shreyas559955/spr25_taxi.mlflow/api/2.0/mlflow/experiments/get failed with exception HTTPSConnectionPool(host='dagshub.com', port=443): Max retries exceeded with url: /Shreyas559955/spr25_taxi.mlflow/api/2.0/mlflow/experiments/get?experiment_id=None (Caused by ResponseError('too many 500 error responses'))

Hyperparameter TUNING USING BEST LEARNING RATE

In [ ]:
import lightgbm as lgb
import mlflow
import mlflow.sklearn
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
import pandas as pd

print("Step 2: Tuning Other Hyperparameters...")
param_distributions = {
    "num_leaves": [10, 20, 40],
    "max_depth": [5, 10, 15],
    "min_child_samples": [2, 5, 10],
    "feature_fraction": [0.6, 0.8],
    "reg_lambda": [0, 0.1, 0.5]
}

model = lgb.LGBMRegressor(learning_rate=best_lr)

random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    n_iter=10,  # Number of combinations to test
    scoring="neg_mean_absolute_error",
    cv=3,
    verbose=2,
    random_state=42,
)

random_search.fit(X_train_only_numeric, y_train)

# Get best parameters and evaluate
best_params = random_search.best_params_
best_model = random_search.best_estimator_
y_pred_final = best_model.predict(X_test_only_numeric)
final_mae = mean_absolute_error(y_test, y_pred_final)

print(f"Final Model MAE: {final_mae}")

# Convert integer columns to float before logging in MLflow
input_example = pd.DataFrame(X_test_only_numeric.iloc[:1]).astype("float64")

# Log final results in MLflow with corrected schema
with mlflow.start_run():
    mlflow.log_params(best_params)
    mlflow.log_metric("final_mae", final_mae)
    mlflow.sklearn.log_model(best_model, "best_lightgbm_model", input_example=input_example)
